In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.svm import SVR
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import KFold, LeaveOneGroupOut

In [2]:
def report_metrics(name, y_true, y_pred):
    """
    Prints RMSE, MAE and R² for the given predictions.
    Args:
        name (str): label for the dataset (e.g. 'Train', 'Val', 'Test').
        y_true (array-like): ground-truth target values.
        y_pred (array-like): model predictions.
    """
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae  = mean_absolute_error(y_true, y_pred)
    r2   = r2_score(y_true, y_pred)
    print(f"{name:25s} | RMSE: {rmse:7.4f} | MAE: {mae:7.4f} | R²: {r2:7.4f}")


In [3]:
grid_size = 1                               # can be 1, 2, 3, 4, or 5
file_prefix = f"{grid_size}x{grid_size}"  

# === Load pre-split datasets ===
df_train = pd.read_csv(f"{file_prefix}_train_apple.csv")
df_test  = pd.read_csv(f"{file_prefix}_test_apple.csv")

print(f"Loaded datasets — Train: {len(df_train)}, Test: {len(df_test)}")


In [4]:
# === Separate features and target variable ===
X_train = df_train.drop(columns=["apple_content"])
y_train = df_train["apple_content"]

X_test  = df_test.drop(columns=["apple_content"])
y_test  = df_test["apple_content"]

In [5]:
# === Feature and target scaling using Min-Max normalization ===
input_scaler  = MinMaxScaler()
target_scaler = MinMaxScaler()

# Fit scalers on training data
X_train_scaled = input_scaler.fit_transform(X_train)
X_test_scaled  = input_scaler.transform(X_test)

y_train_scaled = target_scaler.fit_transform(y_train.values.reshape(-1, 1)).flatten()
y_test_scaled  = target_scaler.transform(y_test.values.reshape(-1, 1)).flatten()


In [ ]:
# === Train SVM regression model ===
svr = SVR(kernel="rbf", C=110, epsilon=0.2, gamma="scale")
svr.fit(X_train_scaled, y_train_scaled)

print(f"✅ SVM trained successfully for grid size: {grid_size}×{grid_size}")

In [7]:
train_preds_scaled = svr.predict(X_train_scaled)
train_preds = target_scaler.inverse_transform(train_preds_scaled.reshape(-1, 1)).flatten()
train_targets = target_scaler.inverse_transform(y_train_scaled.reshape(-1, 1)).flatten()

test_preds_scaled = svr.predict(X_test_scaled)
test_preds = target_scaler.inverse_transform(test_preds_scaled.reshape(-1, 1)).flatten()
test_targets = target_scaler.inverse_transform(y_test_scaled.reshape(-1, 1)).flatten()

In [ ]:
report_metrics("Train", train_targets, train_preds)
report_metrics("Test", test_targets, test_preds)